In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


def preprocess_data(targets, observed, estimated, test):
    """
    Preprocess the data by resampling, merging with targets, and dropping unnecessary columns.
    
    Parameters:
    - targets: Target dataframe with 'time' and target values.
    - observed: Dataframe with observed features.
    - estimated: Dataframe with estimated features.
    - test: Dataframe with test features.
    
    Returns:
    - Preprocessed dataframes ready for training and testing.
    """

    # Ensure the datetime columns are in datetime format
    targets['time'] = pd.to_datetime(targets['time'])
    observed['date_forecast'] = pd.to_datetime(observed['date_forecast'])
    estimated['date_forecast'] = pd.to_datetime(estimated['date_forecast'])
    test['date_forecast'] = pd.to_datetime(test['date_forecast'])

    # Ensure data is sorted by date_forecast
    targets = targets.sort_values(by='time')
    observed = observed.sort_values(by='date_forecast')
    estimated = estimated.sort_values(by='date_forecast')
    test = test.sort_values(by='date_forecast')

    targets = targets[(targets['pv_measurement'] == 0) | (targets['pv_measurement'] != targets['pv_measurement'].shift())]
    targets.dropna(inplace=True)


    """   # Identify boolean columns
    # Forward fill NaNs for boolean columns
    for df in [observed, estimated, test]:
        df[boolean_features] = df[boolean_features].fillna(method='ffill')

    # Forward fill for time-series data (for non-boolean columns)
    for df in [observed, estimated, test]:
        df[df.columns.difference(boolean_features)] = df[df.columns.difference(boolean_features)].fillna(method='ffill') """


    # Resample observed, estimated, and test data to 1 hour using mean() as aggregator
    # and drop rows where all columns are NaN
    observed_resampled = observed.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()
    estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()
    test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()

    """ # Round boolean columns after resampling
    for df in [observed_resampled, estimated_resampled, test_resampled]:
        df[boolean_features] = df[boolean_features].round(0) """

    observed_resampled['estimated'] = 0
    estimated_resampled['estimated'] = 1
    test_resampled['estimated'] = 1
    
    # Merge the observed and estimated data
    weather_data = pd.concat([observed_resampled, estimated_resampled])

    # Merge with target values
    merged_data = pd.merge(targets, weather_data, how='inner', left_on='time', right_on='date_forecast')

    # Time-Based Features (training data)
    merged_data['hour'] = merged_data['date_forecast'].dt.hour
    merged_data['sin_hour'] = np.sin(2 * np.pi * merged_data['hour'] / 23)
    merged_data['cos_hour'] = np.cos(2 * np.pi * merged_data['hour'] / 23)
    merged_data['day_of_week'] = merged_data['date_forecast'].dt.dayofweek
    merged_data['sin_day_of_week'] = np.sin(2 * np.pi * merged_data['day_of_week'] / 7)
    merged_data['cos_day_of_week'] = np.cos(2 * np.pi * merged_data['day_of_week'] / 7)
    merged_data['month'] = merged_data['date_forecast'].dt.month
    merged_data['sin_month'] = np.sin(2 * np.pi * merged_data['month'] / 12)
    merged_data['cos_month'] = np.cos(2 * np.pi * merged_data['month'] / 12)

    # Time-Based Features (test data)
    test_resampled['hour'] = test_resampled['date_forecast'].dt.hour
    test_resampled['sin_hour'] = np.sin(2 * np.pi * test_resampled['hour'] / 23)
    test_resampled['cos_hour'] = np.cos(2 * np.pi * test_resampled['hour'] / 23)
    test_resampled['day_of_week'] = test_resampled['date_forecast'].dt.dayofweek
    test_resampled['sin_day_of_week'] = np.sin(2 * np.pi * test_resampled['day_of_week'] / 7)
    test_resampled['cos_day_of_week'] = np.cos(2 * np.pi * test_resampled['day_of_week'] / 7)
    test_resampled['month'] = test_resampled['date_forecast'].dt.month
    test_resampled['sin_month'] = np.sin(2 * np.pi * test_resampled['month'] / 12)
    test_resampled['cos_month'] = np.cos(2 * np.pi * test_resampled['month'] / 12)
    
    # fixing ceiling_height NaN value
    merged_data['ceiling_height_agl:m'].fillna(0, inplace=True)
    test_resampled['ceiling_height_agl:m'].fillna(0, inplace=True)
    merged_data['cloud_base_agl:m'].fillna(0, inplace=True)
    test_resampled['cloud_base_agl:m'].fillna(0, inplace=True)

    merged_data = merged_data.drop(columns=['time'])
    merged_data.to_csv(f'{loc}_csv/X_train.csv')
    test_resampled.to_csv(f'{loc}_csv/X_test.csv')

    # Drop non-feature columns
    merged_data = merged_data.drop(columns=['date_forecast', 'pv_measurement', 'snow_density:kgm3'])
    test_resampled = test_resampled.drop(columns=['date_forecast', 'snow_density:kgm3'])
    
    return merged_data, test_resampled, targets

locations = ['A', 'B', 'C']
all_predictions = []
all_predictions_rf = []
test_all = pd.DataFrame()

for loc in locations:
    # Load your data
    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')

    # save as csv for analysis
    """ train.to_csv(f'{loc}_csv/train_targets.csv')
    X_train_estimated.to_csv(f'{loc}_csv/X_train_estimated.csv')
    X_train_observed.to_csv(f'{loc}_csv/X_train_observed.csv')
    X_test_estimated.to_csv(f'{loc}_csv/X_test_estimated.csv') """

   # Preprocess data
    X_train, X_test, targets = preprocess_data(train, X_train_observed, X_train_estimated, X_test_estimated)
    targets.to_csv(f'{loc}_csv/processed_targets.csv')
    y = targets['pv_measurement'].values
    test_all = pd.concat([test_all, X_test])


    # Ensure X and y have the same length
    min_length = min(len(X_train), len(y))
    X_train, y_train = X_train.iloc[:min_length], y[:min_length]

    
    X_train_data, X_eval_data, y_train_data, y_eval_data = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Create catboost Pool objects
    #train_pool = Pool(data=X_train_data, label=y_train_data, cat_features=['estimated'])
    #eval_pool = Pool(data=X_eval_data, label=y_eval_data, cat_features=['estimated'])

    # Initialize and Train model
    #categorical_features = ['dew_or_rime:idx', 'elevation:m', 'is_day:idx', 'is_in_shadow:idx', 'snow_drift:idx', 'wind_speed_w_1000hPa:ms']

    #model = CatBoostRegressor(depth=8, learning_rate=0.03, iterations=1000, loss_function='MAE')
    #model.fit(train_pool, use_best_model=True, eval_set=eval_pool)

    model = CatBoostRegressor(loss_function='MAE', learning_rate=0.03, depth=8, cat_features=['estimated'])
    model.fit(X_train, y_train, eval_set=(X_eval_data, y_eval_data))

    # Make predictions using X_test_estimated data
    predictions = model.predict(X_test)

    """ 
    # Define a parameter grid to search over
    param_grid = {
        'depth': [6, 8, 10],  # Example: Try depths of 6, 8, and 10
        'learning_rate': [0.03, 0.1],  # Example: learning rates to try
        # Add other parameters here
    }

    # Create a CatBoostRegressor
    cat_model = CatBoostRegressor(loss_function='MAE', iterations=1000)

    # Instantiate GridSearchCV
    grid_search = GridSearchCV(estimator=cat_model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')

    # Fit the GridSearchCV object with your training data Pool
    grid_search.fit(X_train, y_train)  # You may need to convert your Pool to a dataframe if GridSearchCV doesn't accept Pool objects directly

    # Get the best parameters
    best_parameters = grid_search.best_params_
    print(f"Best parameters: {best_parameters}")

    # Get the best estimator (model with best parameters)
    best_model = grid_search.best_estimator_

    # Use the best model to predict on test data
    best_predictions = best_model.predict(X_test) 
    """
    
    # Store the predictions in all_predictions list
    all_predictions.append(predictions)

    """ 
    # Initialize and Train RandomForest model
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)

    # Make predictions using X_test data
    predictions_rf = model_rf.predict(X_test)
    
    # Store the RandomForest predictions in all_predictions_rf list
    all_predictions_rf.append(predictions_rf) 
    
    final_predictions_rf = np.concatenate(all_predictions_rf)
    
    average_predictions = (np.array(final_predictions) + np.array(final_predictions_rf)) / 2.0
    """
# postprocessing

# Concatenate all predictions
final_predictions = np.concatenate(all_predictions)

# Convert 'final_predictions' to a pandas DataFrame
preds = pd.DataFrame(final_predictions, columns=['prediction'])

test_all.reset_index(drop=True, inplace=True)
preds.reset_index(drop=True, inplace=True)

preds.index = test_all.index

# Setting all night-time predictions to zero
preds.loc[test_all['is_day:idx'] == 0, 'prediction'] = 0

# Save the final_predictions to CSV
df = pd.DataFrame(final_predictions, columns=['prediction'])
df['id'] = df.index
df = df[['id', 'prediction']]
df['prediction'] = df['prediction'].apply(lambda x: max(0, x))
df.to_csv('final_predictions.csv', index=False)


/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:56: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:57: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()


0:	learn: 613.6502902	test: 587.1312631	best: 587.1312631 (0)	total: 18.8ms	remaining: 18.8s
1:	learn: 599.6995600	test: 573.3937418	best: 573.3937418 (1)	total: 32.9ms	remaining: 16.4s
2:	learn: 588.8730237	test: 562.8787042	best: 562.8787042 (2)	total: 50.2ms	remaining: 16.7s
3:	learn: 576.1176519	test: 550.4032835	best: 550.4032835 (3)	total: 64.9ms	remaining: 16.2s
4:	learn: 563.3220693	test: 537.8795239	best: 537.8795239 (4)	total: 78.6ms	remaining: 15.6s
5:	learn: 550.6911518	test: 525.7275396	best: 525.7275396 (5)	total: 92ms	remaining: 15.2s
6:	learn: 537.6855528	test: 513.0613321	best: 513.0613321 (6)	total: 106ms	remaining: 15s
7:	learn: 527.7775392	test: 503.3737072	best: 503.3737072 (7)	total: 120ms	remaining: 14.8s
8:	learn: 515.8983451	test: 491.6644524	best: 491.6644524 (8)	total: 134ms	remaining: 14.8s
9:	learn: 504.9460327	test: 480.8955214	best: 480.8955214 (9)	total: 148ms	remaining: 14.7s
10:	learn: 495.6851640	test: 471.5892981	best: 471.5892981 (10)	total: 162ms	r

/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:56: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:57: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()


0:	learn: 91.9210936	test: 88.2103376	best: 88.2103376 (0)	total: 19.4ms	remaining: 19.4s
1:	learn: 90.1256449	test: 86.4454978	best: 86.4454978 (1)	total: 31.8ms	remaining: 15.9s
2:	learn: 88.5392231	test: 84.8794417	best: 84.8794417 (2)	total: 44.9ms	remaining: 14.9s
3:	learn: 86.9154485	test: 83.3105465	best: 83.3105465 (3)	total: 57.1ms	remaining: 14.2s
4:	learn: 85.2227267	test: 81.6253222	best: 81.6253222 (4)	total: 67.9ms	remaining: 13.5s
5:	learn: 83.4976023	test: 79.8982367	best: 79.8982367 (5)	total: 78.5ms	remaining: 13s
6:	learn: 81.8017168	test: 78.2456307	best: 78.2456307 (6)	total: 91.7ms	remaining: 13s
7:	learn: 79.8521120	test: 76.3677665	best: 76.3677665 (7)	total: 103ms	remaining: 12.8s
8:	learn: 78.1630231	test: 74.7042091	best: 74.7042091 (8)	total: 114ms	remaining: 12.6s
9:	learn: 76.6504491	test: 73.2402873	best: 73.2402873 (9)	total: 125ms	remaining: 12.4s
10:	learn: 74.9787318	test: 71.6300651	best: 71.6300651 (10)	total: 137ms	remaining: 12.3s
11:	learn: 73.47

/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:56: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_15809/980993324.py:57: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(thresh=30).reset_index()


0:	learn: 61.8624782	test: 62.8433770	best: 62.8433770 (0)	total: 18.4ms	remaining: 18.4s
1:	learn: 61.0114935	test: 61.9800649	best: 61.9800649 (1)	total: 30.3ms	remaining: 15.1s
2:	learn: 59.9150315	test: 60.8009665	best: 60.8009665 (2)	total: 41ms	remaining: 13.6s
3:	learn: 58.9772043	test: 59.8193360	best: 59.8193360 (3)	total: 52.8ms	remaining: 13.2s
4:	learn: 58.1000176	test: 58.9376833	best: 58.9376833 (4)	total: 64.1ms	remaining: 12.8s
5:	learn: 57.2446582	test: 58.0582261	best: 58.0582261 (5)	total: 75ms	remaining: 12.4s
6:	learn: 56.2818647	test: 57.0700021	best: 57.0700021 (6)	total: 87.3ms	remaining: 12.4s
7:	learn: 55.4495514	test: 56.2026782	best: 56.2026782 (7)	total: 98.4ms	remaining: 12.2s
8:	learn: 54.6150606	test: 55.3337083	best: 55.3337083 (8)	total: 109ms	remaining: 12s
9:	learn: 53.8369013	test: 54.5288253	best: 54.5288253 (9)	total: 122ms	remaining: 12s
10:	learn: 53.0531670	test: 53.7135048	best: 53.7135048 (10)	total: 133ms	remaining: 12s
11:	learn: 52.2725277

In [2]:
print(test_all)

[     absolute_humidity_2m:gm3  air_density_2m:kgm3  ceiling_height_agl:m  \
0                       4.325              1.28675            912.700012   
1                       4.275              1.28600           1482.099976   
2                       4.150              1.28375           1791.300049   
3                       4.025              1.28200           2312.875000   
4                       3.900              1.28100           2198.299805   
..                        ...                  ...                   ...   
715                     8.350              1.19725           3638.899902   
716                     8.525              1.20050           3552.100098   
717                     8.825              1.20450           2315.000000   
718                     9.025              1.20700           2202.800049   
719                     9.050              1.20775           2015.675049   

     clear_sky_energy_1h:J  clear_sky_rad:W  cloud_base_agl:m  \
0                 0.0